# Nipype interfaces

Nipype interfaces are the core processes that perform operations on data. We create a neuroimaging pipeline by combining these processes.

## Interfaces for third-party neuroimaging toolkits

Nipype already implements a lot of interfaces. Most of them wrap existing command line tools from neuroimaging toolkits, such as

- `nipype.interfaces.afni`
- `nipype.interfaces.fsl`
- `nipype.interfaces.spm`
- `nipype.interfaces.freesurfer`
- `nipype.interfaces.ants`

A list of existing interfaces can be found in the nipype documentation: https://nipype.readthedocs.io/en/latest/interfaces.html

Let's take for example FSL's MCFLIRT, a tool used for motion correction. The help for each interface can be printed with the method `.help()`.

In [1]:
from nipype.interfaces import fsl

fsl.MCFLIRT.help()

Wraps the executable command ``mcflirt``.

FSL MCFLIRT wrapper for within-modality motion correction

For complete details, see the `MCFLIRT Documentation.
<https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/MCFLIRT>`_

Examples
--------
>>> from nipype.interfaces import fsl
>>> mcflt = fsl.MCFLIRT()
>>> mcflt.inputs.in_file = 'functional.nii'
>>> mcflt.inputs.cost = 'mutualinfo'
>>> mcflt.inputs.out_file = 'moco.nii'
>>> mcflt.cmdline
'mcflirt -in functional.nii -cost mutualinfo -out moco.nii'
>>> res = mcflt.run()  # doctest: +SKIP

Inputs::

        [Mandatory]
        in_file: (a pathlike object or string representing an existing file)
                timeseries to motion-correct
                argument: ``-in %s``, position: 0

        [Optional]
        out_file: (a pathlike object or string representing a file)
                file to write
                argument: ``-out %s``
        cost: ('mutualinfo' or 'woods' or 'corratio' or 'normcorr' or
                  'normmi' or 'leastsquares

If we want to use an interface by itself without plugging in into a pipeline, we can follow the example in the docstring.

Note that the following cell will fail, because `functional.nii` does not exist, and inputs and outputs are validated. In this case, the interface requires the input file to exist.

In [3]:
# This will fail
mcflt = fsl.MCFLIRT()
mcflt.inputs.in_file = 'functional.nii'
mcflt.inputs.cost = 'mutualinfo'
mcflt.inputs.out_file = 'moco.nii'
mcflt.cmdline

TraitError: The 'in_file' trait of a MCFLIRTInputSpec instance must be a pathlike object or string representing an existing file, but a value of 'functional.nii' <class 'str'> was specified.

We are just going to create a file called `functional.nii` to skip this error. Note that the check is only about the filename, and not the content of the file.

In [4]:
!touch functional.nii

In [7]:
mcflt = fsl.MCFLIRT()
mcflt.inputs.in_file = 'functional.nii'
mcflt.inputs.cost = 'mutualinfo'
mcflt.inputs.smooth = 3.5
mcflt.inputs.out_file = 'moco.nii'
# This will print the command line argument that will be run
# when executing mcflt.run()
mcflt.cmdline

'mcflirt -in functional.nii -cost mutualinfo -out moco.nii -smooth 3.50'

## Python-based interfaces

A small number of interfaces are based on custom python code shipped with Nipype. The most useful interfaces of this type are used to generate nuisance regressors, or confounds. They are under `nipype.algorithms.confounds`. These three interfaces are very useful.

In [8]:
from nipype.algorithms.confounds import CompCor, FramewiseDisplacement

The `CompCor` interface is used to generate regressors for white-matter detrending (or CompCor):

In [9]:
CompCor.help()

Interface with core CompCor computation, used in aCompCor and tCompCor.

CompCor provides three pre-filter options, all of which include per-voxel
mean removal:

  - ``'polynomial'``: Legendre polynomial basis
  - ``'cosine'``: Discrete cosine basis
  - ``False``: mean-removal only

In the case of ``polynomial`` and ``cosine`` filters, a pre-filter file may
be saved with a row for each volume/timepoint, and a column for each
non-constant regressor.
If no non-constant (mean-removal) columns are used, this file may be empty.

If ``ignore_initial_volumes`` is set, then the specified number of initial
volumes are excluded both from pre-filtering and CompCor component
extraction.
Each column in the components and pre-filter files are prefixe with zeros
for each excluded volume so that the number of rows continues to match the
number of volumes in the input file.
In addition, for each excluded volume, a column is added to the pre-filter
file with a 1 in the corresponding row.

Example
------

The `FramewiseDisplacement` interface is used to compute framewise displacement from motion-correction parameters:

In [10]:
FramewiseDisplacement.help()

Calculate the :abbr:`FD (framewise displacement)` as in [Power2012]_.
This implementation reproduces the calculation in fsl_motion_outliers

.. [Power2012] Power et al., Spurious but systematic correlations in functional
     connectivity MRI networks arise from subject motion, NeuroImage 59(3),
     2012. doi:`10.1016/j.neuroimage.2011.10.018
     <https://doi.org/10.1016/j.neuroimage.2011.10.018>`_.

Inputs::

        [Mandatory]
        in_file: (a pathlike object or string representing an existing file)
                motion parameters
        parameter_source: ('FSL' or 'AFNI' or 'SPM' or 'FSFAST' or 'NIPY')
                Source of movement parameters

        [Optional]
        radius: (a float, nipype default value: 50)
                radius in mm to calculate angular FDs, 50mm is the default since it
                is used in Power et al. 2012
        out_file: (a pathlike object or string representing a file, nipype
                  default value: fd_power_2012.txt)
    

## Writing your own interfaces

Writing a new interface is fairly easy. The hardest part is figuring out the correct "traits" for each input/output parameter, so that they will be validated correctly. 

The easiest way to write a new interface is by copying an existing one and replacing the command line argument (if you are wrapping a command line tool) or the python code that performs whatever operation you need to do. If you are wrapping a command line tool from a neuroimaging toolkit that does not exist in Nipype yet, consider opening a PR to nipype, so others can benefit from your hard work.

From more information about writing your own interfaces, check the detailed developers documentation on Nipype's docs: https://nipype.readthedocs.io/en/latest/devel/interface_specs.html